### 모델 로드

In [1]:
import timm
import torch.nn as nn
model = timm.create_model('vit_base_patch16_224', pretrained=True)



c:\Users\USER\anaconda3\envs\torch_mlp_mixer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pwd

'c:\\Users\\USER\\Desktop'

In [3]:
import torch
import torch.nn as nn
checkpoint = torch.load("C:/Users/USER/Downloads/vit_base_p16_224-80ecf9dd.pth", map_location='cpu') # GPU 환경이 아닌 경우 'cpu'를 사용합니다.
model.load_state_dict(checkpoint)

# 모델의 마지막 분류기 부분을 새로운 클래스 수에 맞게 변경
num_classes = 3  # 새로운 클래스 수
model.head = nn.Linear(model.head.in_features, num_classes)

### 분류기만 학습

In [3]:
# 모델의 모든 파라미터를 동결
for param in model.parameters():
    param.requires_grad = False

# 출력층 파라미터만 학습 가능하도록 설정
model.head.weight.requires_grad = True
model.head.bias.requires_grad = True


In [6]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [ ]:
# 모폴로지 데이터 전처리
import cv2
import numpy as np
import torchvision.transforms as transforms
from torchvision.transforms import functional as F
from PIL import Image

class MorphologyTransform:
    def __call__(self, img):
        # img는 PIL 이미지
        img = np.array(img)
        kernel = np.ones((5,5), np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        return Image.fromarray(img)


In [4]:
import logging
import os
import json
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, SequentialSampler
import cv2
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir,annotations_dir,  transform=None):
        """
        annotation_dir (string): 메타데이터가 있는 JSON 파일의 경로
        img_dir (string): 모든 이미지가 있는 디렉토리의 경로
        transform (callable, optional): 샘플에 적용될 선택적 변환
        """
        self.img_dir = img_dir
        self.transform = transform
        
        self.annotation_dir= annotations_dir
        

    def __len__(self):
        label_list= os.listdir(self.annotation_dir)
        return len(label_list)
    
    def preprocess(self, image_path):
        # 이미지 로드 및 HSV 변환
        with open(image_path, 'rb') as f:
            arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
            image = cv2.imdecode(arr, -1)
        
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # 스킨 색상 범위 설정
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)

        # 스킨 색상 마스크 생성
        skin_mask = cv2.inRange(hsv_image, lower_skin, upper_skin)

        # 마스크를 사용하여 스킨 색상 영역 추출
        skin = cv2.bitwise_and(image, image, mask=skin_mask)

        # OpenCV 이미지를 PIL 이미지로 변환 (transforms와 호환되도록)
        skin_pil = Image.fromarray(cv2.cvtColor(skin, cv2.COLOR_BGR2RGB))

        return skin_pil
    
    def mopology(self, image_path):
        with open(image_path, 'rb') as f:
            arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
            image = cv2.imdecode(arr, -1)

            kernel = np.ones((5,5), np.uint8)
            img = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
            return Image.fromarray(img)

    def __getitem__(self, idx):
        label_list= os.listdir(self.annotation_dir)
        
        img_path = os.path.join(self.img_dir, label_list[idx].split('.')[0]+'.'+label_list[idx].split('.')[1])
        try:
            image = Image.open(img_path)
            preprocessed_image = self.mopology(img_path)
        except (IOError, OSError) as e:
            print(f"Error loading image {img_path}: {e}")
            return self.__getitem__((idx + 1) % len(self))
        # preprocessed_image = self.preprocess(img_path)
        
        
        # faceExp_uploader 부분만 라벨로 사용
        with open(self.annotation_dir+'/'+label_list[idx],'r', encoding='utf-8') as f:
            self.image_labels=json.load(f)
        label = self.image_labels['faceExp_uploader']
        label_to_int = {'기쁨': 0, '당황': 1, '중립': 2, '불안':3}

        # 문자열 라벨을 정수로 매핑
        label_int = label_to_int[label]
        label_tensor = torch.tensor(label_int, dtype=torch.long)
        
        if self.transform:
            image_tensor = self.transform(preprocessed_image)
        else:
            # 기본적으로 이미지를 Tensor로 변환
            transform = ToTensor()
            image_tensor = transform(preprocessed_image)
        
        return image_tensor, label_tensor

In [5]:
def get_loader(img_size, train_img_dir,train_annotation_dir, test_img_dir, test_annotation_dir, train_batch_size, eval_batch_size):

    transform_train = transforms.Compose([
        #주어진 이미지에서 임의의 크기 및 비율로 크롭한 후, 지정된 크기로 이미지를 리사이즈합니다, 5%~100% 사이의 영역을 crop 하여 resizing, crop되는 부분은 랜덤, ex) 중앙하단, 왼쪽 상단 등
        transforms.RandomResizedCrop((img_size, img_size), scale=(0.05, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), # 픽셀에 normalize 적용, 각 채널의 평균과 표준 편차, pre trained된 데이터의 통계를 기반으로 설정
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    trainset = CustomDataset(img_dir=train_img_dir,
                             annotations_dir=train_annotation_dir,
                             transform=transform_train)
    testset = CustomDataset(img_dir=test_img_dir,
                            annotations_dir=test_annotation_dir,
                            transform=transform_test)
    

    train_sampler = RandomSampler(trainset) # 데이터셋에서 무작위로 샘플을 선택,  데이터셋의 인덱스를 무작위로 섞어서 데이터의 순서를 랜덤하게 배치, 모델이 순서에 의존하지 않고 학습함
    test_sampler = SequentialSampler(testset) # 데이터셋에서 순차적으로 샘플을 선택, 데이터를 처음부터 끝까지 순서대로 샘플링
    train_loader = DataLoader(trainset,
                              sampler=train_sampler,
                              batch_size=train_batch_size,
                              num_workers=0,
                              pin_memory=True)
    test_loader = DataLoader(testset,
                             sampler=test_sampler,
                             batch_size=eval_batch_size,
                             num_workers=0,
                             pin_memory=True) if testset is not None else None

    return train_loader, test_loader

In [6]:
#학습 시 하이퍼 파라미터 지정
num_epochs=15
train_batch_size = 64  # 훈련 배치 크기
eval_batch_size = 64  # 평가 배치 크기
train_img_dir= 'C:/Users/USER/Desktop/data/train_img'
train_annotation_dir= 'C:/Users/USER/Desktop/data/train_label'
test_img_dir='C:/Users/USER/Desktop/data/valid_img'
test_annotaion_dir='C:/Users/USER/Desktop/data/valid_label'
name= 'vit_class_3_final_pre'
output_dir= 'output'
eval_every = 113  # 몇 스텝마다 평가를 할 것인지
learning_rate = 3e-2  # 초기 학습률
weight_decay = 0  # 가중치 감소율
sweight_decay = 0  # 가중치 감소율
num_steps = eval_every * num_epochs  # 총 훈련 스텝
seed = 42  # 초기화를 위한 랜덤 시드
n_gpu=1
gradient_accumulation_steps = 1  # 업데이트를 위해 누적할 스텝 수
warmup_steps = 150  # 웜업을 위한 스텝 수
max_grad_norm = 1.0  # 최대 그래디언트 노름

In [7]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import math
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm_
import random
import numpy as np
import logging

from torch.utils.tensorboard import SummaryWriter
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

logger = logging.getLogger(__name__)

#스케줄러
class WarmupCosineSchedule(LambdaLR):
    """ 학습 초기에는 학습률을 점진적으로 증가시키는 warmup기간을 갖고 그 이후에는 코사인 함수를 따라 학습률을 감소시킴
        스케줄러는 optimizer에 설정된 학습률을 기준으로 하여, 특정 시점에서의 학습률을 조절하는 비율(factor)을 계산
        warmup 기간 내: step / warmup_steps 비율로 학습률을 증가, 초기 학습률 x step/warmup_steps
        warmup 기간 이후: 초기 학습률 x 0.5 * (1 + cos(π * cycles * 2 * progress)) 공식에 따라 학습률이 감소
    """
    def __init__(self, optimizer, warmup_steps, t_total, cycles=.5, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        self.cycles = cycles
        super(WarmupCosineSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1.0, self.warmup_steps))
        # progress after warmup
        progress = float(step - self.warmup_steps) / float(max(1, self.t_total - self.warmup_steps))
        return max(0.0, 0.5 * (1. + math.cos(math.pi * float(self.cycles) * 2.0 * progress)))

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def valid(model, writer, test_loader,eval_batch_size, global_step,device):
    # Validation!
    eval_losses = AverageMeter()

    logger.info("***** Running Validation *****")
    logger.info("  Num steps = %d", len(test_loader))
    logger.info("  Batch size = %d", eval_batch_size)

    model.eval()
    all_preds, all_label = [], []
    epoch_iterator = tqdm(test_loader,
                          desc="Validating... (loss=X.X)",
                          bar_format="{l_bar}{r_bar}",
                          dynamic_ncols=True)
    loss_fct = torch.nn.CrossEntropyLoss()
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        x, y = batch
        with torch.no_grad():
            
            logits = model(x)
            eval_loss = loss_fct(logits, y)
            eval_losses.update(eval_loss.item())

            preds = torch.argmax(logits, dim=-1)

        if len(all_preds) == 0:
            all_preds.append(preds.detach().cpu().numpy())
            all_label.append(y.detach().cpu().numpy())
        else:
            all_preds[0] = np.append(
                all_preds[0], preds.detach().cpu().numpy(), axis=0
            )
            all_label[0] = np.append(
                all_label[0], y.detach().cpu().numpy(), axis=0
            )
        epoch_iterator.set_description("Validating... (loss=%2.5f)" % eval_losses.val)

    all_preds, all_label = all_preds[0], all_label[0]
    accuracy = simple_accuracy(all_preds, all_label)
    print("Global Steps: %d" % global_step)
    print("Valid Loss: %2.5f" % eval_losses.avg)
    print("Valid Accuracy: %2.5f" % accuracy)

    logger.info("/n")
    logger.info("Validation Results")
    logger.info("Global Steps: %d" % global_step)
    logger.info("Valid Loss: %2.5f" % eval_losses.avg)
    logger.info("Valid Accuracy: %2.5f" % accuracy)

    writer.add_scalar("test/loss", scalar_value=eval_losses.avg, global_step=global_step)
    writer.add_scalar("test/accuracy", scalar_value=accuracy, global_step=global_step)
    return accuracy

def save_model(model, output_dir, name):
    model_to_save = model.module if hasattr(model, 'module') else model
    model_checkpoint = os.path.join(output_dir, "%s_checkpoint.bin" % name)
    torch.save(model_to_save.state_dict(), model_checkpoint)
    logger.info("Saved model checkpoint to [DIR: %s]", output_dir)

def simple_accuracy(preds, labels):
    return (preds == labels).mean()

 



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

os.makedirs(output_dir, exist_ok=True)
writer = SummaryWriter(log_dir=os.path.join("logs", name))

train_loader, test_loader = get_loader(224, train_img_dir, train_annotation_dir, test_img_dir, test_annotaion_dir, train_batch_size, eval_batch_size)

optimizer = torch.optim.SGD(model.parameters(),
                            lr=learning_rate,
                            momentum=0.9,
                            weight_decay=weight_decay)
t_total = num_steps

# 스케줄러
scheduler = WarmupCosineSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)

model.zero_grad()
# set seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

losses = AverageMeter()
global_step, best_acc = 0, 0

# epoch 단위로 학습 시작
for epoch in range(num_epochs):
    model.train()
    epoch_iterator = tqdm(train_loader,
                          desc=f"Epoch {epoch + 1}/{num_epochs}",
                          bar_format="{l_bar}{r_bar}",
                          dynamic_ncols=True)
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        x, y = batch
        outputs = model(x)
        loss_fn = nn.CrossEntropyLoss()
        #loss 계산
        loss = loss_fn(outputs, y)
        #역전파 학습, 기울기 계산
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0: # 여러 스텝에 걸쳐 gradient를 축적하고 모델에 업데이트, gpu 메모리가 작을 때 큰 배치사이즈를 사용 가능
            #loss update
            losses.update(loss.item() * gradient_accumulation_steps)
            #임계값이 넘는 gradient를 임계값으로 수정, gradient 폭발 방지
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            scheduler.step() # 새로운 학습률을 계산하고 optimizer에 지정되어 있는 학습률 update
            optimizer.step() # weight update
            optimizer.zero_grad() # 이전 gradient 초기화
            global_step += 1

            epoch_iterator.set_description(
                f"Epoch {epoch + 1}/{num_epochs} (Step {global_step} / {t_total}) (Loss={losses.val:.5f})"
            )
            writer.add_scalar("train/loss", scalar_value=losses.val, global_step=global_step)
            writer.add_scalar("train/lr", scalar_value=scheduler.get_lr()[0], global_step=global_step)

            if global_step % eval_every == 0:
                accuracy = valid(model, writer, test_loader, eval_batch_size, global_step, device)

                if best_acc < accuracy:
                    save_model(model, output_dir, name)
                    best_acc = accuracy
                model.train()
    
    losses.reset() # 각 epoch의 끝에 loss 초기화

writer.close()



Epoch 1/15:   0%|| 0/113 [00:00<?, ?it/s]c:\Users\USER\anaconda3\envs\torch_mlp_mixer\lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(
c:\Users\USER\anaconda3\envs\torch_mlp_mixer\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 1/15 (Step 1 / 1695) (Loss=1.13424):   0%|| 0/113 [00:08<?, ?it/s]c

Global Steps: 113
Valid Loss: 0.54975
Valid Accuracy: 0.77889


Validating... (loss=0.91786): 100%|| 29/29 [03:37<00:00,  7.50s/it]00:09,  9.24s/it]


Global Steps: 226
Valid Loss: 0.51118
Valid Accuracy: 0.80000


Validating... (loss=0.60925): 100%|| 29/29 [03:37<00:00,  7.50s/it]00:09,  9.09s/it]


...

Epoch 5/15 (Step 565 / 1695) (Loss=0.61340): 100%|| 113/113 [20:17<00:00, 10.77s/it]


Global Steps: 565
Valid Loss: 0.56080
Valid Accuracy: 0.75000


Validating... (loss=0.80358): 100%|| 29/29 [03:37<00:00,  7.50s/it]00:09,  9.08s/it]


Global Steps: 678
Valid Loss: 0.38312
Valid Accuracy: 0.86222


Epoch 7/15 (Step 702 / 1695) (Loss=0.34751):  21%|| 24/113 [03:33<13:14,  8.93s/it]

### 분류기만 학습

In [8]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import math
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm_
import random
import numpy as np
import logging
from PIL import Image, ImageFile
from torch.utils.tensorboard import SummaryWriter

logger = logging.getLogger(__name__)

ImageFile.LOAD_TRUNCATED_IMAGES = True
#스케줄러
class WarmupCosineSchedule(LambdaLR):
    """ 학습 초기에는 학습률을 점진적으로 증가시키는 warmup기간을 갖고 그 이후에는 코사인 함수를 따라 학습률을 감소시킴
        스케줄러는 optimizer에 설정된 학습률을 기준으로 하여, 특정 시점에서의 학습률을 조절하는 비율(factor)을 계산
        warmup 기간 내: step / warmup_steps 비율로 학습률을 증가, 초기 학습률 x step/warmup_steps
        warmup 기간 이후: 초기 학습률 x 0.5 * (1 + cos(π * cycles * 2 * progress)) 공식에 따라 학습률이 감소
    """
    def __init__(self, optimizer, warmup_steps, t_total, cycles=.5, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        self.cycles = cycles
        super(WarmupCosineSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1.0, self.warmup_steps))
        # progress after warmup
        progress = float(step - self.warmup_steps) / float(max(1, self.t_total - self.warmup_steps))
        return max(0.0, 0.5 * (1. + math.cos(math.pi * float(self.cycles) * 2.0 * progress)))

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def valid(model, writer, test_loader,eval_batch_size, global_step,device):
    # Validation!
    eval_losses = AverageMeter()

    logger.info("***** Running Validation *****")
    logger.info("  Num steps = %d", len(test_loader))
    logger.info("  Batch size = %d", eval_batch_size)

    model.eval()
    all_preds, all_label = [], []
    epoch_iterator = tqdm(test_loader,
                          desc="Validating... (loss=X.X)",
                          bar_format="{l_bar}{r_bar}",
                          dynamic_ncols=True)
    loss_fct = torch.nn.CrossEntropyLoss()
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        x, y = batch
        with torch.no_grad():
            
            logits = model(x)
            eval_loss = loss_fct(logits, y)
            eval_losses.update(eval_loss.item())

            preds = torch.argmax(logits, dim=-1)

        if len(all_preds) == 0:
            all_preds.append(preds.detach().cpu().numpy())
            all_label.append(y.detach().cpu().numpy())
        else:
            all_preds[0] = np.append(
                all_preds[0], preds.detach().cpu().numpy(), axis=0
            )
            all_label[0] = np.append(
                all_label[0], y.detach().cpu().numpy(), axis=0
            )
        epoch_iterator.set_description("Validating... (loss=%2.5f)" % eval_losses.val)

    all_preds, all_label = all_preds[0], all_label[0]
    accuracy = simple_accuracy(all_preds, all_label)
    print("Global Steps: %d" % global_step)
    print("Valid Loss: %2.5f" % eval_losses.avg)
    print("Valid Accuracy: %2.5f" % accuracy)

    logger.info("/n")
    logger.info("Validation Results")
    logger.info("Global Steps: %d" % global_step)
    logger.info("Valid Loss: %2.5f" % eval_losses.avg)
    logger.info("Valid Accuracy: %2.5f" % accuracy)

    writer.add_scalar("test/accuracy", scalar_value=accuracy, global_step=global_step)
    return accuracy

def save_model(model, output_dir, name):
    model_to_save = model.module if hasattr(model, 'module') else model
    model_checkpoint = os.path.join(output_dir, "%s_checkpoint.bin" % name)
    torch.save(model_to_save.state_dict(), model_checkpoint)
    logger.info("Saved model checkpoint to [DIR: %s]", output_dir)

def simple_accuracy(preds, labels):
    return (preds == labels).mean()

 



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

os.makedirs(output_dir, exist_ok=True)
writer = SummaryWriter(log_dir=os.path.join("logs", name))

train_loader, test_loader = get_loader(224, train_img_dir, train_annotation_dir, test_img_dir, test_annotaion_dir, train_batch_size, eval_batch_size)

optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                            lr=learning_rate,
                            momentum=0.9,
                            weight_decay=weight_decay)
t_total = num_steps

# 스케줄러
scheduler = WarmupCosineSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)

model.zero_grad()
# set seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

losses = AverageMeter()
global_step, best_acc = 0, 0

# epoch 단위로 학습 시작
for epoch in range(num_epochs):
    model.train()
    epoch_iterator = tqdm(train_loader,
                          desc=f"Epoch {epoch + 1}/{num_epochs}",
                          bar_format="{l_bar}{r_bar}",
                          dynamic_ncols=True)
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)
        x, y = batch
        outputs = model(x)
        loss_fn = nn.CrossEntropyLoss()
        #loss 계산
        loss = loss_fn(outputs, y)
        #역전파 학습, 기울기 계산
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0: # 여러 스텝에 걸쳐 gradient를 축적하고 모델에 업데이트, gpu 메모리가 작을 때 큰 배치사이즈를 사용 가능
            #loss update
            losses.update(loss.item() * gradient_accumulation_steps)
            #임계값이 넘는 gradient를 임계값으로 수정, gradient 폭발 방지
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            scheduler.step() # 새로운 학습률을 계산하고 optimizer에 지정되어 있는 학습률 update
            optimizer.step() # weight update
            optimizer.zero_grad() # 이전 gradient 초기화
            global_step += 1

            epoch_iterator.set_description(
                f"Epoch {epoch + 1}/{num_epochs} (Step {global_step} / {t_total}) (Loss={losses.val:.5f})"
            )
            writer.add_scalar("train/loss", scalar_value=losses.val, global_step=global_step)
            writer.add_scalar("train/lr", scalar_value=scheduler.get_lr()[0], global_step=global_step)

            if global_step % eval_every == 0:
                accuracy = valid(model, writer, test_loader, eval_batch_size, global_step, device)

                if best_acc < accuracy:
                    save_model(model, output_dir, name)
                    best_acc = accuracy
                model.train()
    
    losses.reset() # 각 epoch의 끝에 loss 초기화

writer.close()



Epoch 1/15:   0%|| 0/113 [00:00<?, ?it/s]c:\Users\USER\anaconda3\envs\torch_mlp_mixer\lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(
c:\Users\USER\anaconda3\envs\torch_mlp_mixer\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 1/15 (Step 1 / 1695) (Loss=1.23820):   0%|| 0/113 [00:06<?, ?it/s]c

Global Steps: 113
Valid Loss: 0.76957
Valid Accuracy: 0.63778


Validating... (loss=0.78176): 100%|| 29/29 [02:53<00:00,  5.97s/it]00:06,  6.87s/it]


Global Steps: 226
Valid Loss: 0.73327
Valid Accuracy: 0.67167


Epoch 3/15 (Step 339 / 1695) (Loss=0.88313): 100%|| 113/113 [15:18<00:00,  8.13s/it]


...

Global Steps: 1469
Valid Loss: 0.63534
Valid Accuracy: 0.71278


Epoch 14/15 (Step 1582 / 1695) (Loss=0.54780): 100%|| 113/113 [15:00<00:00,  7.97s/it]


Global Steps: 1582
Valid Loss: 0.63188
Valid Accuracy: 0.71889


Epoch 15/15 (Step 1695 / 1695) (Loss=0.69411): 100%|| 113/113 [15:00<00:00,  7.97s/it]

Global Steps: 1695
Valid Loss: 0.62623
Valid Accuracy: 0.72222
